In [1]:
import json
import pandas as pd

from individual_players import PossessionAllocator

In [2]:
leagues = ["womens", "mens"]

In [3]:
for league in leagues:
    with open(f"./data/{league}_rosters.json", encoding="utf-8") as file:
        rosters = json.load(file)

    player_ratings = pd.read_csv(f"./data/{league}_player_ratings.csv", index_col=0)
    player_info = pd.read_csv(f"./data/{league}_player_info.csv", index_col="player_id")
    player_ratings = player_ratings.merge(
        player_info, left_index=True, right_index=True, how="left"
    )
    defense_ratings = pd.read_csv(
        f"./data/{league}_player_ratings_defense.csv", index_col=0
    )
    player_ratings = player_ratings.merge(
        defense_ratings, left_index=True, right_index=True, suffixes=("", "_defense")
    ).assign(net_vpp=lambda _: _.vpp - _.vpp_defense)
    teams = (
        player_info.reset_index()[["team_name", "team_id"]]
        .drop_duplicates()
        .set_index("team_id")
    )

    allocator = PossessionAllocator.load(f"./models/{league}_allocator.pkl")

    # TODO: move some of this stuff into team objects to tidy it up a bit,
    # allow more "interative" work like looking into matchups
    team_ratings = {}
    active_players = set()
    team_player_ratings = {}
    for team_id, player_ids in rosters.items():
        team_player_ratings[team_id] = {
            pid: (
                player_ratings.at[pid, "net_vpp"],
                player_ratings.at[pid, "vpp_var"],
            )
            for pid in player_ids
        }
        possession_props = allocator.allocate(team_player_ratings[team_id])
        active_players |= team_player_ratings[team_id].keys()
        team_ratings[team_id] = sum(
            team_player_ratings[team_id][player][0] * weight
            for player, weight in possession_props.items()
        )

    print(f"\n\n\n{league=}")
    for team_id, rating in sorted(
        team_ratings.items(), key=lambda tr: tr[1], reverse=True
    ):
        link = f"https://www.espn.com/{league}-college-basketball/team/_/id/{team_id}"
        try:
            team_name = teams.at[team_id, "team_name"]
        except KeyError:
            team_name = "?"
        print(f"{rating:.03f}\t{team_name}\t{link}")
        ranked_players = sorted(
            team_player_ratings[team_id].items(), key=lambda pr: pr[1][0], reverse=True
        )
        for player_id, (vpp, vpp_var) in ranked_players:
            try:
                player_name = player_info.at[player_id, "player_name"]
            except KeyError:
                player_name = f"https://www.espn.com/{league}-college-basketball/player/_/id/{player_id}"
            print(f"\t{vpp:.03f}\t{player_name}")

    player_ratings[[i in active_players for i in player_ratings.index]].sort_values(
        "net_vpp", ascending=False
    )

    for i, row in enumerate(
        player_ratings[[i in active_players for i in player_ratings.index]]
        .sort_values("net_vpp", ascending=False)
        .itertuples()
    ):
        print(
            f"{i}\t{row.net_vpp:.03f}\t{row.player_name:<30}\t{row.team_name:<30}\thttps://www.espn.com/{league}-college-basketball/player/_/id/{row.Index}"
        )





league='womens'
0.165	south-carolina-gamecocks	https://www.espn.com/womens-college-basketball/team/_/id/2579
	0.229	Kamilla Cardoso
	0.215	https://www.espn.com/womens-college-basketball/player/_/id/4432831
	0.160	Sania Feagin
	0.144	https://www.espn.com/womens-college-basketball/player/_/id/4565516
	0.139	https://www.espn.com/womens-college-basketball/player/_/id/4398752
	0.108	Zia Cooke
	0.103	https://www.espn.com/womens-college-basketball/player/_/id/4281185
	0.091	Talaysia Cooper
	0.090	https://www.espn.com/womens-college-basketball/player/_/id/4432834
	0.088	https://www.espn.com/womens-college-basketball/player/_/id/4432835
	0.068	Raven Johnson
	-0.016	https://www.espn.com/womens-college-basketball/player/_/id/4433380
0.136	texas-longhorns	https://www.espn.com/womens-college-basketball/team/_/id/251
	0.183	Taylor Jones
	0.175	Aaliyah Moore
	0.150	https://www.espn.com/womens-college-basketball/player/_/id/4400473
	0.138	Khadija Faye
	0.136	DeYona Gaston
	0.120	https://www.espn.co